In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pysrim import dose, Ion, Layer

In [ ]:
ion = Ion('H', energy=34.7e6)

layer = Layer(
    elements={
        'Al': {
            'stoich':   1.0,
            'E_d':     25.0, # Displacement energy [eV], common default
            'lattice':  0.0, # Lattice binding energy [eV]
            'surface': 3.36  # Surface binding energy [eV]
        }
    },
    density=2.702,          # Aluminium density in g/cm^3
    width=3.0e7,             # 1 mm in Angstroms
    name="Aluminium"
)

res_E = dose.residual_energy(ion, layer)
print(f"Residual energy after traversing through {layer.name} layer {layer.width:.2e} Angstroms thick: {res_E:.2f} MeV")

In [ ]:
depths_mm = np.linspace(0,10,100)
residual_energies = dose.residual_energy(ion, layer, depths_mm)
plt.plot(depths_mm, residual_energies)
plt.xlabel("Depth in Al (cm)")
plt.ylabel("Proton Energy (MeV)")
plt.title("Proton Energy vs. Depth in Aluminum")
plt.grid(True)
plt.show()

In [ ]:
srim_executable_directory = '/Users/mdovale/Applications/srim' 

ion = Ion('H', energy=50e6)

layer = Layer(
    elements={
        'Al': {
            'stoich':   1.0,
            'E_d':     25.0, # Displacement energy [eV], common default
            'lattice':  0.0, # Lattice binding energy [eV]
            'surface': 3.36  # Surface binding energy [eV]
        }
    },
    density=2.702,          # Aluminium density in g/cm^3
    width=1.0e7,             # 1 mm in Angstroms
    name="Aluminium"
)

import copy
stack = [copy.deepcopy(layer) for _ in range(3)]

dose.residual_energy_through_stack(ion, stack, verbose=True)

In [ ]:
ion = Ion('H', energy=45e6)

layer = Layer(
    elements={
        'Al': {
            'stoich':   1.0,
            'E_d':     25.0, # Displacement energy [eV], common default
            'lattice':  0.0, # Lattice binding energy [eV]
            'surface': 3.36  # Surface binding energy [eV]
        }
    },
    density=2.702,          # Aluminium density in g/cm^3
    width=3.0e7,             # 1 mm in Angstroms
    name="Aluminium"
)

dose.compute_tid(ion, [layer], 3e10, verbose=True)

In [ ]:
al_widths = np.arange(100) / 10  # 0 to 9.9 mm in 0.1 mm steps
fluence = 1e10  # ions/cm²

res_E, doses = dose.sweep_al_shield_dose(
    al_thicknesses_mm=al_widths,
    fluence=fluence,
)

In [ ]:
# Create figure and axis
fig, ax1 = plt.subplots(figsize=(9, 6))

# Plot Dose on left y-axis (red)
ax1.set_xlabel("Aluminum thickness (mm)", fontsize=12)
ax1.set_ylabel("Dose (rad Si)", color='red', fontsize=12)
ax1.plot(al_widths, doses, 'r-', linewidth=2.5, label="Dose")
ax1.tick_params(axis='y', labelcolor='red')
ax1.set_ylim(0, max(doses)*1.1)

# Plot Residual Energy on right y-axis (blue)
ax2 = ax1.twinx()
ax2.set_ylabel("Residual energy (MeV)", color='blue', fontsize=12)
ax2.plot(al_widths, res_E, 'b-', linewidth=2.5, label="Residual energy")
ax2.tick_params(axis='y', labelcolor='blue')
ax2.set_ylim(0, 50)

# Annotate fluence and beam energy
ax1.text(
    0.5, 0.95,
    r"$10^{10}$ protons/cm$^2$ @ 45 MeV",
    transform=ax1.transAxes,
    ha='center', va='top',
    fontsize=13,
    bbox=dict(facecolor='white', edgecolor='black')
)

# Combine legends from both axes
lines = ax1.get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='center left', edgecolor='black', fancybox=True, shadow=True, framealpha=1)

# Grid and axis ticks
ax1.grid(True, axis='x', linestyle='--', linewidth=0.5)
ax1.set_xticks(np.arange(0, 11, 1))
ax1.set_xlim(0, 10)

# Title and layout
plt.title("Dose and Residual Energy vs. Aluminum Shielding", fontsize=14)
plt.tight_layout()
plt.show()